In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


전체 여행 로그 데이터

In [9]:
%cd /content/drive/MyDrive/Colab Notebooks/Capstone Design 2023 2nd/DropoutNet/torch

/content/drive/MyDrive/Colab Notebooks/Capstone Design 2023 2nd/DropoutNet/torch


In [12]:
!ls

data.py		    model.py	 rating_matrix.ipynb  utils.py
main_travel_log.py  __pycache__  test_torch_model.py


In [19]:
!python main_travel_log.py --data-dir ../Dataset/all_travel_log/ --checkpoint-path ../checkpoint/ --tb-log-path ../tb_log --model-device "cuda"

running: 2023-11-28-05:51:17_2.0
[main] loaded U:(11687, 200),V:(9717, 200) elapsed [0 s]
[main] standardized U,V elapsed [0 s]
[main] loaded user feature sparse matrix: (11687, 6) elapsed [0 s]
[main] read train triplets 63784 elapsed [0 s]
[utils] read eval_cold triplets (7416,) elapsed [0 s]
[utils] loaded eval_cold elapsed [0 s]
	n_test_users:[5203]
	n_test_items:[7416]
	R_train_inf: no R_train_inf for cold
	R_test_inf: shape=(5203, 7416) nnz=[7191]
[main] initialized numpy data elapsed [0 s]
[main] initialized eval_cold elapsed [0 s]
	u_concat rank=206
	v_concat rank=200
[MAIN EPOCH : 1]
ubatch:   0% 0/20000 [00:00<?, ?it/s]/content/drive/MyDrive/Colab Notebooks/Capstone Design 2023 2nd/DropoutNet/torch/model.py:110: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Ucontent = torch.tensor(Ucontent, dtype=torch.float32)
/content/drive/

수도권 여행 로그 데이터 결과

## 2023.10.12-11:08
- 실행 후 약 2분 40초 후 정지. 코드 상 main.py의 66줄 이후까지 결과 확인됨.
- prep eval 부분을 하고 끝나는지 그 전에 마치는지 봐야함

## 2023.10.12-11:23
- prep eval에서 timer.tic()이후 종료되는 것 같음
- 71라인의 init_tf에 문제가 있을 것으로 예상

## 2023.10.13-14:13
- Colab 기준 코드 실행시 GPU RAM 의 변화량 없음
- GPU 설정이 잘 되었는지 확인하기

## 2023.10.13-14:44
- 일단 main.py의 eval_warm.init_tf에서 실행이 안되므로 해당 부분 코드 print()로 확인

## 2023.10.13-14:58
- 높은 확률로 Colab의 메모리부족으로 인한 문제인 것으로 추정

## 2023.10.13-15:18
- DropoutNet이 데이터를 어떻게 불러오는지 파악할 예정
- 첫 번째 문제: init_tf에서의 메모리 문제 -> todense 실행 중 종료
- V_content_test 데이터가 어디로부터 시작되는지 파악해야 함

## 2023.10.13-15:22
- V_content_test = item_content\[self.test_item_ids, :]
- item_content 인자: init_tf의 4번째 인자
- self.test_item_ids 인자: EvalData 생성자의 2번째 인자로부터 얻어짐
- eval_warm = dat\['eval_warm'] 값임
- eval_warm.init_tf()실행하므로 eval_warm은 EvalData 클래스이며 eval_batch_size가 5번째 인자 값으로 들어감(5 번째를 4 번째 인자로 착각해서 찾음...)
- 초기 eval_batch_size = 1000으로 설정

## 2023.10.13-15:28
- 초기 eval_batch_size의 값을 줄여보는 방식으로 코드 실행 예정

## 2023.10.13-15:41
- user_batch_size, eval_batch_size 각각 200으로 줄였을 때 실패
- 각각 값을 32로 초기화하고 재실행
- 32도 실패시 2로 설정해보고 진행
- 그래도 실패시 data를 불러오고 저장한 값들을 generator 형식으로 변형한 후 진행할 예정

## 2023.10.13-15:45
- batch_size가 관여하는 코드 가기 전 todense에서 RAM을 12.7GB 찍고 종료됨
- 이전에 메모리 관리를 할 방도를 고려해야함

## 2023.10.13-15:48
- item_content_파일, 확인 요망

## 2023.10.13-16:01
- Item_content 변수의 원래 파일은 item_features_0based.txt이다.
- 해당 파일은 libsvm format의 파일

## libsvm format 파일의 형태
- DATA_NUM feature_num:value, feature_num2:value ... feature_numN:value
- 데이터 번호 이후 여러 feature에 대한 value값들로 구성됨

## 2023.10.13-16:14
- 현재 진행상황에 대한 결론
- DropoutNet 참 좋은 모델인 것은 맞음
- 그러나 사용 데이터가 오픈되지 않음
- 따라서 관련된 데이터로부터 vector값으로 이미 처리됨
- 논문을 읽고 어떻게 vector와 했는지 감이 잡히고 실행 가능하면 해당 모델로 쭉 가도 될 것 같음
- but 어떻게 libsvm, 등의 파일의 값을 추출했는지에 대한 설명이 없으면 불가

## 2023.10.13-19:47
- 논문의 Experiments 부분 읽은 후 CiteULike dataset으로 모델 돌려보기로 결정
- CiteULike의 User/Item의 개수가 훨씬 적음(각각 5,551/16,980 개)
- Github 설명따라 main_cold_citeu.py 실행, but 오류 발생
- 266줄의 pd.read_csv()중 header 값에서 오류
```
ValueError: Passing negative integer to header is invalid. For no header, use header=None instead
```
- 불러오는 파일의 헤더 확인 필요

## 2023.10.13-19:54
```python
split_folder = os.path.join(data_path, 'cold')
train_file = os.path.join(split_folder, 'train.csv')
```
- 해당 경로 /eval/cold/train.csv 확인
- train.csv는 총 \{"A", "B", "C"} 3개의 column으로 구성
- 각각의 column의 의미 파악 필요

## 2023.10.13-20:08
- train.csv는 User-Item pair에 대한 matrix

## 2023.10.13-20:30~21:17
- 기존 train_file 불러오는 코드 수정 후 오류 해결
- 이후 test_cold_file 불러오는 코드 오류, 해결
- 이후 data.py 124줄의 undefined xrange 변수 오류 발견, range로 변경
```
AttributeError: module 'tensorflow' has no attribute 'ConfigProto'
```
- 이후 해당 오류 발견
- tf.compat.v1.ConfigProto()코드로 변경
```
AttributeError: module 'tensorflow' has no attribute 'placeholder'
```
- 해당 오류:tf1 오류 tf2 되면서 사라짐
- tf.compat.v1.placeholder로 변경해 사용 유지
- 대부분의 코드 tf2의 model.py 참고해 실행

## 2023.10.13-21:49
- main_cold_citeu.py 성공적으로 돌아가는 것 확인
- 훈련이 잘 마치고도 에러 없는지 확인해야함
- 현재 epoch 돌면서 훈련 진행 중














